In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Decision Tree Income Prediction

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('/kaggle/input/income/adult_dataset.csv')
df.head()

In [4]:
df.info()

In [5]:
df1 = df[df.workclass == '?']
df1.info()

In [6]:
df = df[df.workclass != '?']
df.head()

In [7]:
df.shape

In [8]:
df_catagorial = df.select_dtypes(include=['object'])

df_catagorial.apply(lambda x: x == '?', axis = 0).sum()

In [9]:
df = df[df.occupation != '?']

df = df[df['native.country'] != '?']

df.info()

# Label Encoding

In [10]:
df_catagorial = df.select_dtypes(include=['object'])

df_catagorial.head()

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_catagorial = df_catagorial.apply(le.fit_transform)

df_catagorial.head()

In [12]:
df = df.drop(df_catagorial.columns, axis=1)

df = pd.concat([df, df_catagorial], axis=1)
df.head()

In [13]:
df.info()

In [14]:
df['income'] = df['income'].astype('category')
df.head()

In [15]:
df.info()

In [16]:
from sklearn.model_selection import train_test_split

x = df.drop('income', axis=1)

y = df['income']

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=101)

In [17]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth = 5)

tree.fit(xtrain, ytrain)

In [18]:
from sklearn.metrics import classification_report, accuracy_score

ypredictions = tree.predict(xtest)

In [19]:
print(classification_report(ytest, ypredictions))

In [20]:
accuracy_score(ytest,ypredictions)

# Hyperparameter Tuning

In [21]:
help(DecisionTreeClassifier)

In [23]:
from sklearn.model_selection import GridSearchCV

nfold = 5

parameters = { 'max_depth':range(1,40) }

dtree = DecisionTreeClassifier(criterion = 'gini', random_state=101)

t = GridSearchCV(dtree, parameters, cv=nfold, scoring='accuracy' )

t.fit(xtrain, ytrain)

In [24]:
scores = t.cv_results_
scores = pd.DataFrame(scores)
scores

max_depth = 7

## Tuning min_sample_leaf

In [25]:
nfold = 5

parameters = { 'min_samples_leaf':range(5,200,20) }

dtree = DecisionTreeClassifier(criterion = 'gini', random_state=101)

t = GridSearchCV(dtree, parameters, cv=nfold, scoring='accuracy' )

t.fit(xtrain, ytrain)

In [26]:
scores = t.cv_results_
scores = pd.DataFrame(scores)
scores

In [ ]:
min_samples_leaf = 65

In [27]:
## minimum samples split

In [28]:
nfold = 5

parameters = { 'min_samples_split':range(5,200,20) }

dtree = DecisionTreeClassifier(criterion = 'gini', random_state=101)

t = GridSearchCV(dtree, parameters, cv=nfold, scoring='accuracy' )

t.fit(xtrain, ytrain)

In [29]:
scores = t.cv_results_
scores = pd.DataFrame(scores)
scores

In [ ]:
min_samples_split = 125

In [30]:
parameter_grid = {
    'max_depth':range(5,15,5),
    'min_samples_leaf':range(50,150,50),
    'min_samples_split':range(50,150,50),
    'criterion':['entropy', 'gini']
}

nfold = 5


dtree = DecisionTreeClassifier()

t = GridSearchCV(dtree, param_grid = parameter_grid, cv=nfold, scoring='accuracy' )

t.fit(xtrain, ytrain)

In [31]:
scores = t.cv_results_
scores = pd.DataFrame(scores)
scores

In [32]:
t.best_score_

In [35]:
tree = DecisionTreeClassifier(criterion='gini',
                             random_state=101,
                             max_depth=10,
                             min_samples_leaf=100,
                             min_samples_split=50)

tree.fit(xtrain,ytrain)

In [36]:
tree.score(xtest,ytest)